    
# ECON320-3 Assignment 6: James (Minkyu) Song 


In [ ]:
# Script name: james_song_assignment_06.ipynb
# Purpose of script: Assignment 6 of ECON320 with Dr.O'Connell
# Author: James (Minkyu) Song
# Date Created: October 4th 2022
# --------------------------
# Notes: NA
# ---------------------------


# Assignment 6

## Part I

* Objective: Report the partial effects in a linear regression model.*

* Reference: Wooldridge (7th edition), Chapter 3, Section 2*

Following the previous assignment, take a sample of 1000 observations and estimate a linear model that accords with the population model.

Are the estimates close or different? Then, perform the partialling out approach for $\beta_1$ and $\beta_2$. Show that you achieve the same coefficients as in the Python command that you chose to perform the linear regression (different packages such as statsmodels or sklearn can perform linear regressions).


In [1]:
from ECON320_lib import birthday_dgm, rm_components
# I have set up a library (folder) where there are functions that are often reused for this ECON320 class.
# This library will be subject to change as the course progresses.

In [2]:
import math

import pandas as pd
import numpy as np
import statsmodels.formula.api as smf


Taking a sample of 1000 observations from the previous assignment.

In [3]:
birthdaystring = "04022000" 
rng = np.random.RandomState(int(birthdaystring))

In [4]:
mrm_df, pop_b0_mrm, pop_b1_mrm, pop_b2_mrm = birthday_dgm.generate_mrm_birhtday(birthdaystring, rng, 10, 2, 0, 1000000)
print(f"population b_0 is: {pop_b0_mrm}")
print(f"population b_1 is: {pop_b1_mrm}")
print(f"population b_2 is: {pop_b2_mrm}")

mrm_df[:10]

x_1 is normally distributed with a mean of 10 and variance of 4
x_2 is normally distributed with a mean of 2 and variance of 4
u is normally distributed with a mean of 0 and variance of 2
y = 2 + 4*x_1 + 2*x_2 + u 
population b_0 is: 2
population b_1 is: 4
population b_2 is: 2


,x_1,x_2,u,y
0,9.762285,2.948933,-0.093010,46.853994
1,11.905537,-1.254199,0.595572,47.709323
2,10.940033,-0.759498,1.659478,45.900615
3,12.577342,-0.346016,-2.824877,48.792459
4,12.484329,1.833474,0.227080,55.831345
5,10.995418,4.361952,-0.996175,53.709400
6,11.222126,2.552896,0.848635,52.842931
7,12.043087,1.407567,-1.015456,51.972025
8,12.150970,2.285533,-1.563660,53.611287
9,9.325411,1.279933,0.024420,41.885929


In [5]:
round(mrm_df.describe(),3)

,x_1,x_2,u,y
count,1000000.000,1000000.000,1000000.000,1000000.000
mean,10.001,2.003,0.002,46.012
std,2.000,1.998,1.414,9.061
min,0.452,-7.795,-6.854,3.334
25%,8.651,0.658,-0.952,39.912
50%,10.001,2.004,0.000,45.997
75%,11.349,3.351,0.957,52.127
max,18.997,11.361,6.708,88.592


In [6]:
sample_mrm_df_1k = mrm_df.sample(n = 10000, random_state= int(birthdaystring))

Estimating a linear model that accords with the population model.

In [7]:
reg = smf.ols(formula= 'y ~ x_1 + x_2', data= sample_mrm_df_1k)
result = reg.fit()

print(f'Regression results model 1: \n{result}\n')
result.summary()

Regression results model 1: 



<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.975
Model:                            OLS   Adj. R-squared:                  0.975
Method:                 Least Squares   F-statistic:                 1.967e+05
Date:                Tue, 04 Oct 2022   Prob (F-statistic):               0.00
Time:                        19:16:08   Log-Likelihood:                -17735.
No. Observations:               10000   AIC:                         3.548e+04
Df Residuals:                    9997   BIC:                         3.550e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      1.8543      0.075     24.863      0.000       1.708       2.000
x_1            4.0122      0.007    558.522      0.000       3.998       4.026
x_2            2.0050      0.007    281.095      0.000       1.991       2.019
==============================================================================
Omnibus:                        1.088   Durbin-Watson:                   2.010
Prob(Omnibus):                  0.580   Jarque-Bera (JB):                1.103
Skew:                          -0.025   Prob(JB):                        0.576
Kurtosis:                       2.988   Cond. No.                         54.9
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [8]:
# this will be added to the library
def partial_beta_12_21(df, regressed_var_name, var_name):
    new_df= df.copy(deep=True)
    mod_12_21 = smf.ols(formula=f'{regressed_var_name} ~ {var_name}', data=df)
    res_12_21 = mod_12_21.fit()
    predicted_x1_hat_a = res_12_21.predict()

    samp_x1_x2_beta1 = res_12_21.params[1]
    samp_x1_x2_beta0 = res_12_21.params[0]
    predicted_x1_hat_b = samp_x1_x2_beta0 + samp_x1_x2_beta1*new_df[var_name]
    if ((predicted_x1_hat_a.round(2) == predicted_x1_hat_b.round(2)).all()):
        new_df[f'{regressed_var_name}_hat'] = predicted_x1_hat_a
    else:
        raise ValueError("partialed out x1_hat is inconsistent")
    new_df[f'{regressed_var_name}_residual'] = res_12_21.resid
    return new_df, res_12_21


In [9]:
partialed_sample_mrm_1k, res_x1_x2 = partial_beta_12_21(sample_mrm_df_1k, 'x_1', 'x_2')
partialed_sample_mrm_1k, res_x2_x1 = partial_beta_12_21(partialed_sample_mrm_1k, 'x_2', 'x_1')


In [16]:
result.params[1].round(2)

4.01

In [10]:
mody1 = smf.ols(formula = 'y ~ x_1_residual', data = partialed_sample_mrm_1k)
resy1 = mody1.fit()

resy1.params.round(2)

Intercept       46.05
x_1_residual     4.01
dtype: float64

In [17]:
result.params[2].round(2)

2.0

In [18]:
mody2 = smf.ols(formula = 'y ~ x_2_residual', data = partialed_sample_mrm_1k)
resy2 = mody2.fit()
resy2.params.round(2)

Intercept       46.05
x_2_residual     2.00
dtype: float64

In [ ]:
# As we can see from the initial DGP, 
# population b_0 is: 2
# population b_1 is: 4
# population b_2 is: 2 
# 
# using pandas and numpy we take a sample 
# and build MLRMs using stasmodels
#
# result.summary() table shows us
# sample b_0 is: 1.8543
# sample b_1 is: 4.0122
# sample b_2 is: 2.0050
#
# partialed out method showed us that
# when we partial out sample_b_1, we get the same coefficients of sample b_1 which is 4.01
# when we partial out sample_b_2, we get the same coefficients of sample b_2 which is 2.00

# this is verified by fitting the residual to y both ways which returned the same result of 4.01 and 2.00
# while there are little differences between the population and the sample coefficients, 
# the difference is extremely small, and we can say that our sample and our models are created well. 
